First, we must construct our environment...

In [2]:
import numpy as np
import matplotlib
from rl_glue import RLGlue
from Agent import BaseAgent 
from Environment import BaseEnvironment  
from manager import Manager

In [ ]:
# Like shown previously, lets declare first, then implement
class CaptureGoEnvironment(BaseEnvironment):
    def env_init(self, env_info=...):
        return NotImplementedError
    
    def env_start(self):
        raise NotImplementedError

    def env_step(self, action):
        raise NotImplementedError

    def env_cleanup(self):
        raise NotImplementedError